# Eat Safe, Love

## Notebook Set Up

In [4]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [5]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [6]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [7]:
# review the collections in our database
db.list_collection_names()

['establishments']

In [8]:
# assign the collection to a variable
establishments = db['establishments']
print (establishments)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'uk_food'), 'establishments')


## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [11]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene':20}

# Use count_documents to display the number of documents in the result
print(f"The number of establishments with a hygiene score of 20 is: {establishments.count_documents(query)}")


The number of establishments with a hygiene score of 20 is: 0


In [12]:

# Display the first document in the results using pprint
result = establishments.find_one(query)

print("The first document in the result is: ")
pprint(result)

The first document in the result is: 
None


In [13]:
# Convert the result to a Pandas DataFrame
hygiene20 = establishments.find(query)

hygiene20_df = pd.DataFrame(hygiene20)

# Display the number of rows in the DataFrame
print(f"The number of rows in the DataFrame is: {hygiene20_df.shape[0]}")

# Display the first 10 rows of the DataFrame
hygiene20_df.head(10)

The number of rows in the DataFrame is: 0


""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [15]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query2 = {
    'LocalAuthorityName': {'$regex': 'London'},
    'RatingValue': {'$gte': 4}
}

# Use count_documents to display the number of documents in the result
print(f"The number of establishments with London as the Local Authority and a RatingValue >= to 4 is: {establishments.count_documents(query2)}")


The number of establishments with London as the Local Authority and a RatingValue >= to 4 is: 0


In [16]:

# Display the first document in the results using pprint
result2 = establishments.find_one(query2)

print("The first document in the result is: ")
pprint(result2)

The first document in the result is: 
None


In [17]:
# Convert the result to a Pandas DataFrame
london = establishments.find(query2)

london_df = pd.DataFrame(london)

# Display the number of rows in the DataFrame
print(f"The number of rows in the DataFrame is: {london_df.shape[0]}")

# Display the first 10 rows of the DataFrame
london_df.head(10)

The number of rows in the DataFrame is: 0


""


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [19]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

# Extract the 'geocode' field from the document with 'BusinessName' 'Penang Flavours'
lat_long = establishments.find_one({"BusinessName":"Penang Flavours"},{"geocode":1})

degree_search = 0.01
# Assign the lat and long values to variables
latitude = lat_long['geocode']['latitude']
longitude = lat_long['geocode']['longitude']

query3 = {"geocode.latitude":{'$gte': latitude - degree_search, '$lte': latitude + degree_search},
        "geocode.longitude":{'$gte': longitude - degree_search, '$lte': longitude + degree_search},
        "RatingValue":5}
sort = [("scores.Hygiene",1)]
limit = 5

# Print the results
results3 = list(establishments.find(query).sort(sort).limit(limit))
for r in results3:
    pprint(r)


In [20]:
# Convert result to Pandas DataFrame
near_penang_df = pd.DataFrame(results3)

near_penang_df

""


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [22]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
match = {'$match': {"scores.Hygiene":0}}

group = {'$group': {"_id" : "$LocalAuthorityName",
                   "count": {'$sum':1}}}

sort = {'$sort':{"count":-1}}

pipeline = [match, group, sort]

results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print(f"There are {len(results)} documents in the result.")

# Print the first 10 results
pprint(results[0:10])


There are 0 documents in the result.
[]


In [23]:
# Convert the result to a Pandas DataFrame
hygiene0_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"The number of rows in the DataFrame is: {hygiene0_df.shape[0]}")

# Display the first 10 rows of the DataFrame
hygiene0_df.head(10)

The number of rows in the DataFrame is: 0


""
